## State of Washington - Licensee - Green Freedom

* UBI: 603350787

We'll be using the [`cannapy`](https://github.com/CannabisData/cannapy) library to access the portal data.  `cannapy` aims to provide an abstract interface for accessing and working with *Cannabis* data from around the world.  It utilizes [xmunoz](https://github.com/xmunoz)'s [`sodapy`](https://github.com/xmunoz/sodapy) client to access Socrata-based open data portals and can return data loaded into [Pandas DataFrames](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

### Dataset: Licensed Businesses

* Canonical Dataset ID: **bhbp-x4eb**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/Licensing/Licensed-Businesses/u3zh-ri66
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/bhbp-x4eb

In [9]:
import time
import cannapy.us.wa.wslcb.portal as wslcb
import pandas as pd

In [10]:
# Specify your own Socrata App Token if you plan to experiment
app_token = 'XaB9MBqc81C3KT4Vps6Wh5LZt'

# Instantiate a cannapy interface to the WSLCB open data portal
portal = wslcb.WSLCBPortal(app_token)

# We'll be using the Licensed Businesses dataset
dataset_id = 'bhbp-x4eb'

# And we're looking for data on a particular licensee
licensee_ubi = '603350787'

In [11]:
# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Thu Dec 14 16:37:18 2017


In [12]:
# Retrieve the dataset preloaded into a Pandas DataFrame
licenses = portal.get_dataframe(dataset_id)

In [13]:
# The UBI column uniquely identifies each licensee, but obscures ownership of multiple licenses by the same entity.
# Let's break that column apart into its constituent parts:
# Unified Business Identifier (UBI): first nine digits
# Business ID Number: next three digits
# Location Number: last four digits
df_v2 = licenses.rename(columns={'ubi': 'ubi_source'})
df_v2['ubi'] = df_v2.ubi_source.str[0:9]
df_v2['ubi_business_id'] = df_v2.ubi_source.str[9:12]
df_v2['ubi_location'] = df_v2.ubi_source.str[12:]
licensee_licenses = df_v2.loc[df_v2['ubi'] == licensee_ubi]
licensee_licenses

,license,type,createdate,active,organization,address,address_line_2,city,state,zip,county,dayphone,ubi_source,ubi,ubi_business_id,ubi_location
1741,412515,MARIJUANA PRODUCER TIER 3/MARIJUANA PROCESSOR,20171027,ACTIVE (ISSUED),GREEN FREEDOM,1087 MONTE ELMA RD,NaN,ELMA,WA,985419378,GRAYS HARBOR,3603040166,6033507870010001,603350787,001,0001


### Dataset: Enforcement Visits

* Canonical Dataset ID: **w7wg-8m52**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/dataset/Enforcement-Visits-Dataset/jizx-thwg
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/w7wg-8m52

In [14]:
# Let's see how many enforcement visits the licensee has hosted
dataset_id = 'w7wg-8m52'

# Select the licensee's license number
# TODO: find a way to do this without hardcoding the row number
licensee_license_number = licensee_licenses.loc[1741, 'license']

# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Tue Jan 16 10:35:57 2018


In [15]:
# Retrieve the dataset preloaded into a Pandas DataFrame
enforcement_visits = portal.get_dataframe(dataset_id)

# Suppress the chained assignment warning: https://stackoverflow.com/a/20627316/7622699
pd.options.mode.chained_assignment = None

# Pull aside the enforcement visits by the selected licensee
licensee_enforcement_visits = enforcement_visits.loc[enforcement_visits['license_number'] == licensee_license_number]

# Sort the DataFrame by 'date'
licensee_enforcement_visits.sort_values(by='date', inplace=True)

licensee_enforcement_visits

,date,license_number,city_name,county_name,activity
9840,2015-10-01T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check
11986,2015-12-02T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check
10831,2016-04-12T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check
5109,2016-06-23T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check
7651,2016-06-27T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check
15437,2016-07-14T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check
12709,2017-02-28T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check
16381,2017-03-29T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check
1269,2017-07-18T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check
3736,2017-12-28T00:00:00.000,412515,UNINCORPORATED AREAS,GRAYS HARBOR,Marijuana Premises Check


### Dataset: Violations

* Canonical Dataset ID: **dgm4-3cm6**
* Detail screen on the WSLCB Portal: https://data.lcb.wa.gov/dataset/Violations-Dataset/dx3i-tzh2
* Detail screen on Socrata's Open Data Foundry: https://dev.socrata.com/foundry/data.lcb.wa.gov/dgm4-3cm6

In [16]:
# Let's pull up all of the licensee's violations
dataset_id = 'dgm4-3cm6'

# Check when the dataset was last updated
last_updated = portal.dataset_last_updated(dataset_id)
print('Last updated: {}'.format(time.strftime('%c', last_updated)))

Last updated: Tue Jan 16 10:35:24 2018


In [17]:
# Retrieve the dataset preloaded into a Pandas DataFrame
violations = portal.get_dataframe(dataset_id)

# Pull aside the violations by the selected licensee
licensee_violations = violations.loc[violations['license_number'] == licensee_license_number]

# Sort the DataFrame by 'visit_date'
licensee_violations.sort_values(by='visit_date', inplace=True)

licensee_violations

,visit_date,license_number,county_name,city_name,case,violation_code,wac_code,penalty_type
144,2015-12-02T00:00:00.000,412515,GRAYS HARBOR,UNINCORPORATED AREAS,7D5336A,NaN,314.55.083(4),Written Warning
